# Overview

While Spark is a distributed compute cluster is does provide a nuce feature: we can run the code locally so that we can sebug issues etc.

In this notebook we will run a locak spark cluster for the purposes of debugging in a simpler local environment.

It assumes you have already read the following notebooks:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Spark Pi - The Hello World Example For Apache spark](Spark%20Pi%20-%20The%20Hello%20World%20Example%20For%20Apache%20spark.ipynb)

# 1. Add Spark To The Path

In [1]:
import findspark
findspark.init()

In [2]:
import sys
print(sys.path)

['/usr/lib/spark-3.1.1-bin-hadoop2.7/python', '/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip', '/ml-training-jupyter-notebooks/Machine Learning/Big Data And Big Compute/Apache Spark', '/usr/local/lib/python39.zip', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/site-packages']


# 2. Create The SparkConf Object
The SparkConf object is a configuration object. It configures the SparkContext object and allows us to create and connect to our local cluster.

In [3]:
import pyspark

In [4]:
sparkConf = pyspark.SparkConf()
sparkConf.setAppName("spark-jupyter-win")

# 3. Create the SparkContext
The SparkContext is the object that allows us to interface with apache spark.

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
sc = spark.sparkContext

22/01/15 17:47:10 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 15.4.12.12 instead (on interface eth0)


# 4. Load The Data

In [7]:
# Import the utilities module we wrote
import importlib.util
spec = importlib.util.spec_from_file_location("utilities", "../../../Utilities/utilities.py")
utilities = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utilities)

# Define a mapping to convert our data field to the correct type
converter_mapping = {
    "date": utilities.convert_date_string_to_date
}

In [10]:
from databricks import koalas
koalas_dataframe = koalas.read_csv("../../../Example Data Sets/nasdaq_2019.csv", converters=converter_mapping)
koalas_dataframe.head()

,ticker,interval,date,open,high,low,close,volume
0,AABA,D,2019-07-01,70.90,71.5200,70.3250,70.57,10234800
1,AAL,D,2019-07-01,33.14,33.6632,32.5301,32.88,8995100
2,AAME,D,2019-07-01,2.43,2.4300,2.4000,2.40,500
3,AAOI,D,2019-07-01,10.70,10.8900,10.0100,10.18,883100
4,AAON,D,2019-07-01,50.57,50.9850,48.5600,49.73,180200


# 5. Cleanup The SparkContext
We will stop the local cluster and cleanup our memory objects.

In [11]:
sc.stop()